In [1]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [2]:
!ls *.geojson

delaware_river_basin.geojson


In [3]:
my_aoi_file = 'delaware_river_basin.geojson'

In [4]:
from notebookLib.nb_display import Ldisplay
ld = Ldisplay()
ld.map_geojson(my_aoi_file)

creating Ldisplay class
